In [1]:
import h5py

In [12]:
with h5py.File('Datasets/MNIST/train.hdf5') as f:
    X_train = f['X'][:]
    y_train = f['y'][:]
with h5py.File('Datasets/MNIST/test.hdf5') as f:
    X_test = f['X'][:]
    y_test = f['y'][:]

In [10]:
X_train.shape

(55000, 784)

In [11]:
y_train.shape

(55000, 10)

In [14]:
X_test.shape

(10000, 784)

In [15]:
y_test.shape

(10000, 10)

In [49]:
import numpy as np

In [47]:
def get_next_batch(X, y, size):
    m = X.shape[0]
    index = np.random.permutation(np.arange(m))
    for i in range(m//size):
        batch_i = index[i*size: i*size+size]
        yield X[batch_i], y[batch_i]

In [26]:
import tensorflow as tf

In [34]:
Learning_Rate = 0.001

In [52]:
tf.reset_default_graph()

In [53]:
def conv_layer(input, size_in, size_out):
    initializer = tf.contrib.layers.xavier_initializer()
    w = tf.Variable(initializer([5, 5, size_in, size_out]))
    b = tf.Variable(tf.constant(0.1, shape=[size_out]))
    
    conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
    act = tf.nn.relu(conv + b)
    
    pooled = tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

    return pooled

In [54]:
def fc_layer(input, size_in, size_out):
    initializer = tf.contrib.layers.xavier_initializer()
    w = tf.Variable(initializer([size_in, size_out]))
    b = tf.Variable(tf.constant(0.1, shape=[size_out]))
    
    act = tf.matmul(input, w) + b

    return act

In [55]:
X = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [56]:
X_reshape = tf.reshape(X, shape=[-1, 28, 28, 1])

In [57]:
conv1 = conv_layer(X_reshape, 1, 32)
conv_out = conv_layer(conv1, 32, 64)

In [58]:
flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

fc1 = fc_layer(flattened, 7 * 7 * 64, 1024)
relu = tf.nn.relu(fc1)
logits = fc_layer(relu, 1024, 10)

In [59]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))

train_step = tf.train.AdamOptimizer(Learning_Rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [62]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1, 101):
        for batch in get_next_batch(X_train, y_train, 64):
            _ = sess.run(train_step, {X:batch[0], y:batch[1]})
        if epoch % 10 == 0:
            test_acc, test_loss = sess.run([accuracy, loss], {X:X_test, y:y_test})
            batch = get_next_batch(X_train, y_train, 1000).__next__()
            train_acc, train_loss = sess.run([accuracy, loss], {X:batch[0], y:batch[1]})
            print("Epoch", epoch, "Train Acc:", np.round(train_acc, 5), \
                  "Train Loss:", np.round(train_loss, 5), \
                  "Test Acc:", np.round(test_acc, 5), "Test Loss:", np.round(test_loss, 5))

Epoch 1 Train Acc: 0.979 Train Loss: 0.07407 Test Acc: 0.9767 Test Loss: 0.07228
Epoch 11 Train Acc: 1.0 Train Loss: 0.00287 Test Acc: 0.9889 Test Loss: 0.03965
Epoch 21 Train Acc: 1.0 Train Loss: 0.00074 Test Acc: 0.989 Test Loss: 0.04618
Epoch 31 Train Acc: 0.999 Train Loss: 0.00132 Test Acc: 0.9895 Test Loss: 0.05119
Epoch 41 Train Acc: 0.999 Train Loss: 0.00214 Test Acc: 0.9903 Test Loss: 0.06044
Epoch 51 Train Acc: 1.0 Train Loss: 0.0 Test Acc: 0.9896 Test Loss: 0.07561
Epoch 61 Train Acc: 0.996 Train Loss: 0.02094 Test Acc: 0.9868 Test Loss: 0.11478
Epoch 71 Train Acc: 0.999 Train Loss: 0.00112 Test Acc: 0.99 Test Loss: 0.08779
Epoch 81 Train Acc: 1.0 Train Loss: 0.00074 Test Acc: 0.989 Test Loss: 0.11636
Epoch 91 Train Acc: 0.999 Train Loss: 0.001 Test Acc: 0.99 Test Loss: 0.12093
